In [159]:
import pandas as pd
import numpy as np
import copy
from tqdm import tqdm
from Bio import SeqIO, SearchIO 

In [160]:
rodeo = pd.read_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/main_co_occur_rodeo_psi.csv', sep = ",")
rodeo = rodeo.loc[:,['Query','Protein_acc']]
rodeo

,Query,Protein_acc
0,WP_145853129.1,WP_187365872.1
1,WP_145853129.1,WP_187365873.1
2,WP_145853129.1,WP_145853125.1
3,WP_145853129.1,WP_145853126.1
4,WP_145853129.1,WP_145853127.1
...,...,...
11152,WP_015759610.1,WP_015759614.1
11153,WP_015759610.1,WP_015759615.1
11154,WP_015759610.1,WP_015759616.1
11155,WP_015759610.1,WP_015759617.1


In [161]:
file = '/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/parsed_hmm_per_dom_tbl.txt'
res_domains = dict()
x = 10
count = 0
for res in SearchIO.parse(file, 'hmmscan3-domtab'):
        res_domains[res.id] = {}
        for hit in res.hits:
            if hit.id[:4] == 'TIGR':
                    call = hit.description.split(":")[0]
            else:
                    call = hit.id
            for hsp in hit.hsps:
                overlap = False
                if res_domains[res.id].keys(): #not empty
                    names = copy.deepcopy(list(res_domains[res.id].keys()))
                    for name in names: #проверка на перекрытие с раннее добавленными генами 
                        for q_range in res_domains[res.id][name]:
                            if min(hsp.env_end, q_range[1]) - max(hsp.env_start, q_range[0]) > x:
                                overlap = True
                                break
                        if overlap:
                            break
                    if not overlap: 
                        if call not in res_domains[res.id]:
                            res_domains[res.id][call] = [(hsp.env_start, hsp.env_end)]
                            
                        if (hsp.env_start, hsp.env_end) not in res_domains[res.id][call]:
                                        #для случая мультидоменного белка с одинаковыми доменами: 
                                        #один и тот же белок выравнивается на несколько мест
                            res_domains[res.id][call].append((hsp.env_start, hsp.env_end))
                       
                else:
                    res_domains[res.id][call] = [(hsp.env_start, hsp.env_end)]
                    

In [162]:
domains_names = {}
for key, value in res_domains.items():
    domains_names[key] = list(value.keys())

domains_names = pd.DataFrame({'domains': domains_names.values()}, index =res_domains.keys())
domains_names

,domains
WP_011566059.1,[MnhB]
WP_011566060.1,[2a6301s01]
WP_011566061.1,"[mycofact_glyco, Methyltransf_23, Methyltransf..."
WP_011566062.1,[Creatininase]
WP_011566063.1,[actino_HemFlav]
...,...
WP_160900598.1,"[DoxX_3, DoxX]"
WP_160901386.1,[DNA_alkylation]
WP_160900597.1,"[Pyr_redox_2, Reductase_C]"
WP_160900596.1,"[FAM70, DUF3040]"


In [152]:
domains_count = []
for domain in domains_names['domains']:
    domains_count += domain

domains_count = {k:0 for k in domains_count}

for query in tqdm(rodeo['Query'].unique()):
    temp = rodeo[rodeo['Query'] == query]
    for prot_id in temp['Protein_acc']:
        if prot_id in domains_names.index:
            for domains in domains_names.loc[prot_id]:
                 for name in domains:
                    domains_count[name] += 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 681/681 [00:04<00:00, 168.16it/s]


In [153]:
dom_count = pd.DataFrame({'counts': domains_count.values()} , index = domains_count.keys())
dom_count = dom_count[dom_count['counts'] > 0].sort_values(by = ['counts'], ascending=False)
dom_count

,counts
mycofactocin,668
Mycofactocin_RRE,667
mycofact_rSAM,654
mycofact_TetR,495
actino_HemFlav,493
...,...
thiS,1
HSCB_C,1
antidote_HigA,1
HTH_37,1


In [155]:
#dom_count.to_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/domains_counts_region_10.csv')

---

In [167]:
operon_prot_id = pd.read_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/operon_100_protIDs_rodeo.csv', sep = ",", index_col = 0)
operon_prot_id

,Query,Protein_acc
1,WP_145853129.1,WP_246115648.1
2,WP_145853129.1,WP_246115687.1
3,WP_145853129.1,WP_145853128.1
4,WP_145853129.1,WP_145853129.1
5,WP_145853129.1,WP_145853130.1
...,...,...
5243,WP_015759610.1,WP_015759614.1
5244,WP_015759610.1,WP_015759615.1
5245,WP_015759610.1,WP_015759616.1
5246,WP_015759610.1,WP_015759617.1


In [168]:
domains_count_operon = []
for domain in domains_names['domains']:
    domains_count_operon += domain

domains_count_operon = {k:0 for k in domains_count_operon}

for query in tqdm(operon_prot_id['Query'].unique()):
    temp = operon_prot_id[operon_prot_id['Query'] == query]
    for prot_id in temp['Protein_acc']:
        if prot_id in domains_names.index:
            for domains in domains_names.loc[prot_id]:
                 for name in domains:
                    domains_count_operon[name] += 1
        

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 675/675 [00:02<00:00, 332.78it/s]


In [169]:
dom_count_op = pd.DataFrame({'counts': domains_count_operon.values()} , index = domains_count_operon.keys())
dom_count_op = dom_count_op[dom_count_op['counts'] > 0].sort_values(by = ['counts'], ascending=False)
dom_count_op

,counts
Mycofactocin_RRE,661
mycofact_rSAM,619
mycofactocin,589
actino_HemFlav,356
mycofact_creat,315
...,...
TPR_16,1
Cyclase,1
T7SS_ESX_EspC,1
DUF350,1


In [170]:
#dom_count_op.to_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/domains_counts_operon_100.csv')